In [153]:
import pandas as pd
import os
from datetime import datetime, timedelta

config_file_path = 'Config.csv'

In [154]:
def set_flags(x,limit_type,r_limit,y_limit):
    flag = 'G'
    if limit_type == 'UCL':
        if x >= r_limit:
            flag = 'R'
        elif x>= y_limit:
            flag = 'Y'
    else:
        if x <= r_limit:
            flag = 'R'
        elif x<= y_limit:
            flag = 'Y'        
    return flag

def load_prod_data(server,product,details,limits,debug=False):
    tech = details['TECH']
    prod = details['PART']
    rev = details['REV']
    idv_unit = details['IDV_UNIT']
    sicc_unit = details['SICC_UNIT']
    cap_unit = details['CAPABILITY_UNIT']
    cdyn_unit = details['CDYN_UNIT']
    
    
    folders = r'\Actuals\Last_49_Days'
    if tech=='P1273':
        folders = r'\Actuals\Last_49Days'
    
    file_path = os.path.join(r'\\'+server,tech+r'_Data'+folders,prod+'.csv')
    
    # debug = True
    df = pd.read_csv(file_path)
    # if debug: print(df.head())

    if debug:
        col_check = 1
        num_rows = df.shape[0]
        print(f"Column check {col_check} = {num_rows}")
        col_check +=1
        
    df = df[df['PROCESS_REV'] == rev]

    if debug:
        num_rows = df.shape[0]
        print(f"Column check {col_check} = {num_rows}")
        col_check +=1

    df['SORT_DATE'] = pd.to_datetime(df['SORT_DATE'], errors='coerce')
    # Calculate the start and end dates for the last 4 full weeks
    today = datetime.today()
    start_of_this_week = today - timedelta(days=today.weekday() + 1)
    start_of_4th_last_full_week = start_of_this_week - timedelta(weeks=4)

    # Filter the DataFrame for the last 4 full weeks
    df = df[
        (df['SORT_DATE'] >= start_of_4th_last_full_week)
    ]
    if debug:
        num_rows = df.shape[0]
        print(f"Column check {col_check} = {num_rows}")
        col_check +=1

    possible_columns = ['IDV', 'SICC', 'CAPABILITY', 'CDYN']
    columns_of_interest = []
    
    for col in possible_columns:        
        if pd.notna(details[col]) and details[col] != '':
            columns_of_interest.append(col)
            df[col] = df[details[col]]
    
    means = df.groupby('FAB')[columns_of_interest].mean().reset_index()
    means['TECH'] = tech
    means['PRODUCT'] = product
    # print(means)

    # for col in columns_of_interest:
    #     if col == 'SICC':
    #         mult = 1000
    #         if sicc_unit != 'mA': mult=1
    #         means['SICC'] = means['SICC']*mult
    #         digits = 2
    #         # means['SICC'] = means['SICC'].round(2)
    #     elif col == 'CDYN':
    #         digits = 2
    #     else:
    #         if idv_unit == 'Mhz':
    #             digits = 0
    #         else:
    #             digits = 2
    #     means[col] = means[col].round(digits)
            

 

        
    
    # for col in columns_of_interest:
        # means[col+'_Flag'] = 'Green'
        
    for col in columns_of_interest:
    # print(col)
        limit_type = limits[col+'_TYPE']
        target = details[col+'_TGT']
        r_limit = target*(1+limits[col+'_RED'])
        y_limit = target*(1+limits[col+'_YELLOW'])
        means[col+'_FLAG'] = means[col].apply(set_flags, args=(limit_type, r_limit, y_limit))
    

    
    return means
    
    

In [155]:
def load_excel_to_dict(file_path,key_col):
    df = pd.read_csv(file_path)
    temp_dict = {}
    for _, row in df.iterrows():
        key = row[key_col]
        temp_dict[key] = row.drop(key_col).to_dict()
    return temp_dict

In [156]:
prod_dict = load_excel_to_dict('Config.csv','PRODUCT')
limit_dict = load_excel_to_dict('Tech_Limits.csv','TECH')

In [157]:
# prod_dict

In [158]:
server = 'rasinkul-desk'

temp_dfs = []
for product, details in prod_dict.items():
# if True:
#     product = 'RPL68'
    # details = prod_dict['RPL68']
    print(product)
    limits = limit_dict[details['TECH']]
    # product = details['PART']

    temp_df = load_prod_data(server=server, product=product, details=details,limits=limits, debug=False)

    temp_dfs.append(temp_df)
    
final_df = pd.concat(temp_dfs, ignore_index=True)


ADL PCH-P
RPL816
RPL68
SPRXCC


C:\Users\dagarcia\AppData\Local\Temp\ipykernel_104344\1270321126.py:32: DtypeWarning: Columns (25,29,31) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


GNRXCC


C:\Users\dagarcia\AppData\Local\Temp\ipykernel_104344\1270321126.py:32: DtypeWarning: Columns (459,460,461,462,463,464) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


GNRHCC
GNRLCC


In [159]:
desired_order = ['TECH','PRODUCT','FAB','IDV','SICC','CAPABILITY','CDYN','IDV_FLAG','SICC_FLAG','CAPABILITY_FLAG','CDYN_FLAG']
    
final_df = final_df[desired_order]
final_df

,TECH,PRODUCT,FAB,IDV,SICC,CAPABILITY,CDYN,IDV_FLAG,SICC_FLAG,CAPABILITY_FLAG,CDYN_FLAG
0,P1273,ADL PCH-P,F24,9477.116830,0.094950,9468.519546,NaN,G,G,G,NaN
1,P1274,RPL816,F28,16657.593451,0.034375,16602.661586,3.503019,G,G,G,G
2,P1274,RPL816,F32,16630.425952,0.031276,16686.804273,3.492266,G,G,G,G
3,P1274,RPL68,F28,16410.494936,0.016369,16264.526434,3.258256,G,Y,G,G
4,P1274,RPL68,F32,16236.505135,0.014656,16200.858789,3.251949,G,G,G,G
5,P1274,SPRXCC,F32,14869.755802,13.409435,14953.439309,2.780011,G,G,G,G
6,P1276,GNRXCC,D1D,1.090863,1.010227,1.089291,2.083003,G,G,G,G
7,P1276,GNRXCC,F34,1.103361,1.051874,1.094764,2.084701,G,G,G,G
8,P1276,GNRHCC,D1D,1.063877,0.811468,1.096166,2.086833,R,G,G,G
9,P1276,GNRHCC,F34,1.102391,1.080629,1.089226,2.095432,G,Y,G,G


In [161]:
final_df.to_csv('output.csv')

In [162]:
import numpy as np
from openpyxl import Workbook
from openpyxl.styles import Font, Alignment

In [163]:
wb = Workbook()
ws = wb.active

In [164]:
bold_font = Font(bold=True)
blue_font = Font(color="0000FF")
green_font = Font(color="00B050")
orange_font = Font(color="FF9900")
center_alignment = Alignment(horizontal="center", vertical="center")

In [165]:
ws['A1'] = 'Benchmark/Goal'
ws['A1'].font = bold_font
ws['A1'].alignment = center_alignment

ws['B1'] = 'F32'
ws['B1'].font = bold_font
ws['B1'].alignment = center_alignment

ws['C1'] = 'F28'
ws['C1'].font = bold_font
ws['C1'].alignment = center_alignment


In [166]:
prod_dict['RPL816']['IDV_UNIT']

'Mhz'

In [167]:
def get_value(df, prod_dict, product, fab, column):
    unit = prod_dict[product][column+'_UNIT']
    
    mult = 1
    if column == 'SICC':
        mult = 1000
        if unit != 'mA': mult=1
        # means['SICC'] = means['SICC']*mult
        digits = 2
    elif column == 'CDYN':
        digits = 2
    else:
        if unit == 'Mhz':
            digits = 0
        else:
            digits = 2
    if unit == "NONE": unit = ""
    print(unit)
    print(mult)
    print(digits)
    
    values = df[(df['PRODUCT'] == product) & (df['FAB'] == fab)][column].values
    value = values[0]*mult
    rnd_value = np.round(value, decimals=digits)
    formatted_value = f"{rnd_value:.{digits}f}{unit}"
    return formatted_value

def get_flag(df, product, fab, column):
    flag = df[(df['PRODUCT'] == product) & (df['FAB'] == fab)][column].values
    return flag[0] if len(flag) > 0 else None

In [170]:
get_value(final_df, prod_dict, 'GNRHCC', 'F34','SICC')

A
1
2


'1.08A'

In [ ]:
rows = [
    ['IDV > Target-0.5%',
     f""]
]